In [90]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [92]:
df = pd.read_csv("/content/drive/My Drive/DataSet/Housing.csv",sep=",")
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [93]:
df["furnishingstatus"].unique()

array(['furnished', 'semi-furnished', 'unfurnished'], dtype=object)

In [0]:
## basement,hotwareheating,Airconditioning is just nominal categorical ,guestRoom,Not A RANKING,Right?
## So we are going to turn this to Dummies 
df["mainroad"].replace({"yes" :1 , "no" : 0},inplace=True)
df["guestroom"].replace({"yes" :1 , "no" : 0},inplace=True)
df["basement"].replace({'yes' : 1, "no" : 0 },inplace=True)
df["hotwaterheating"].replace({"yes":1 , "no":0},inplace=True)
df["airconditioning"].replace({"yes":1 , "no":0},inplace=True)
df["prefarea"].replace({"yes":1 , "no" :0},inplace=True)
df["furnishingstatus"].replace({"furnished":2 , "semi-furnished":1, "unfurnished":0},inplace=True)
df["TotalRooms"] = df["bedrooms"].astype(int) + df["bathrooms"].astype(int) +df["guestroom"].astype(int) + df["basement"].astype(int)
df["HasParking"] = np.where(df["parking"] > 1 ,1,0)
df["LotBed"] = np.where(df["bedrooms"] >= 4,1,0)
df["LotBathrooms"] = np.where(df["bathrooms"] >= 3 ,1,0)

In [95]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus,TotalRooms,HasParking,LotBed,LotBathrooms
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2,6,1,1,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2,8,1,1,1
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1,6,1,0,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2,7,1,1,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2,7,1,1,0


In [0]:
x_features = df.drop("price",axis=1)
target = df['price']

In [0]:
## Check The Type of Problems 
#sns.pairplot(df)

In [0]:
def normalize (x): 
    return ( (x-np.min(x))/ (max(x) - min(x)))
                                            
                                              
# applying normalize ( ) to all columns 
df[x_features.columns] = df[x_features.columns].apply(normalize) 

In [0]:
from sklearn.model_selection import GridSearchCV , KFold ,train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor



# hyperparams = {
#                "min_samples_split" : np.arange(1,20,2),
#                "min_samples_leaf" :[1,3,5],
#                "max_depth" : np.arange(1,15,1)}
kfold = KFold(n_splits=8,shuffle=True,random_state=42)
# Model = GridSearchCV(RandomForestRegressor(random_state=42),
#                      param_grid = hyperparams,
#                      verbose = True,
#                      cv = kfold,
#                      n_jobs = -1)

# Model.fit(x_features,target)


In [0]:
df['price'] = np.log(df["price"])

In [101]:
import statsmodels.api as sm
x = sm.add_constant(x_features)
linearreg = sm.OLS(target,x).fit()
print(linearreg.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     75.25
Date:                Thu, 28 May 2020   Prob (F-statistic):          1.67e-120
Time:                        11:21:05   Log-Likelihood:                -8332.3
No. Observations:                 545   AIC:                         1.670e+04
Df Residuals:                     529   BIC:                         1.677e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -3.778e+05   3.19e+05  

Well this is not really linear problem ,seems it's more likely Non Linear Problems

In [102]:
from sklearn.linear_model import ElasticNet
alpha = {"alpha":np.arange(0.01,5,0.01)}
Model_Elastic = GridSearchCV(ElasticNet(),param_grid=alpha,
                             cv=kfold,
                             verbose=True,
                             n_jobs=-1)
Model_Elastic.fit(x_features,target)


Fitting 8 folds for each of 499 candidates, totalling 3992 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 584 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 3992 out of 3992 | elapsed:   13.5s finished


GridSearchCV(cv=KFold(n_splits=8, random_state=42, shuffle=True),
             error_score=nan,
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': array([0.01, 0.02, 0.03...
       4.41, 4.42, 4.43, 4.44, 4.45, 4.46, 4.47, 4.48, 4.49, 4.5 , 4.51,
       4.52, 4.53, 4.54, 4.55, 4.56, 4.57, 4.58, 4.59, 4.6 , 4.61, 4.62,
       4.63, 4.64, 4.65, 4.66, 4.67, 4.68, 4.69, 4.7 , 4.71, 4.72, 4.73,
       4.74, 4.75, 4.76, 4.77, 4.78, 4.79, 4.8 , 4.81, 4.82, 4.83, 4.84,
       4.85, 4.86, 4.87, 4.88, 4.89, 4.9 , 4.91, 4.92, 4.93, 4.94, 4.95,
       4.96, 4.97, 4.98, 4.99])},
             pre_dispatch='2*n_jobs', 

In [103]:
Model_Elastic.best_score_
Model_Elastic.best_params_

{'alpha': 0.05}

In [104]:
x_train,x_test,y_train,y_test = train_test_split(x_features,target,test_size=0.2)
Model_ela = ElasticNet(alpha=0.05)
Model_ela.fit(x_train,y_train)
y_pred = Model_ela.predict(x_test)
from sklearn.metrics import mean_squared_error as mse
mse(y_test,y_pred)

1055640233575.0232

In [105]:
from sklearn.linear_model import LinearRegression
Model_linear = LinearRegression()
Model_linear.fit(x_train,y_train)
y_pred = Model_linear.predict(x_test)
from sklearn.metrics import mean_squared_error as mse
mse(y_test,y_pred)

1042703218232.813

In [106]:
rf = RandomForestRegressor(n_estimators=100,max_depth = 12 , min_samples_leaf = 1, min_samples_split = 15)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
from sklearn.metrics import mean_squared_error as mse
mse(y_test,y_pred)

1025519722232.1195

In [107]:
np.sqrt(mse(y_test,y_pred))
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x_features.values, i) for i in range(x_features.shape[1])]
vif["features"] = x_features.columns

/usr/local/lib/python3.6/dist-packages/statsmodels/stats/outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [108]:
x_features = df.drop(["bedrooms","bathrooms",'guestroom',"basement","TotalRooms","mainroad"],axis=1)
target = df['price']

x_train,x_test,y_train,y_test = train_test_split(x_features,target,test_size=0.2)
Model_ela = ElasticNet(alpha=0.05)
Model_ela.fit(x_train,y_train)
y_pred = Model_ela.predict(x_test)
from sklearn.metrics import mean_squared_error as mse
np.sqrt(mse(y_test,y_pred))

0.11549015827740199

In [109]:
rf = RandomForestRegressor(n_estimators=100,max_depth = 12 , min_samples_leaf = 1, min_samples_split = 15)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
from sklearn.metrics import mean_squared_error as mse
np.sqrt(mse(y_test,y_pred))

0.022042436432266332

In [0]:
#LOL
